# This is a Small Tour about Probabilistic Graphical Models using pgmpy

Ensure these versions are installed:
- conda install pandas=0.18.1 numpy=1.11.1 networkx=1.11 scipy=0.18.0  jupyter
- conda install -c ankurankan pgmpy
- conda install jupyter

Verify occasionally on https://github.com/pgmpy/pgmpy

## Restaurant CPDs
#### Borrowed from the excellent book *Mastering Probabilistic Graphical Models Using Python* from Abinash Panda, Ankur Ankan

#### Verifications and tests made with Samiam: http://reasoning.cs.ucla.edu/samiam/
![restaurant_CPDs](restaurant_CPDs.png)

In [33]:
import numpy as np
import pandas as pd
import datetime
from pgmpy.models import BayesianModel
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD
# Now first create the model.
restaurant = BayesianModel([('location', 'cost'),
                            ('quality', 'cost'),
                            ('cost', 'no_of_people'),
                            ('location', 'no_of_people')])
cpd_location = TabularCPD('location', 2, [[0.6, 0.4]])
cpd_quality = TabularCPD('quality', 3, [[0.3, 0.5, 0.2]])
cpd_cost = TabularCPD('cost', 2,
                      [[0.8, 0.6, 0.1, 0.6, 0.6, 0.05],
                       [0.2, 0.4, 0.9, 0.4, 0.4, 0.95]],
                      ['location', 'quality'], [2, 3])
cpd_no_of_people = TabularCPD('no_of_people', 2,
                              [[0.6, 0.8, 0.1, 0.6],
                               [0.4, 0.2, 0.9, 0.4]],
                              ['location', 'cost'], [2, 2])
restaurant.add_cpds(cpd_location, cpd_quality,
                    cpd_cost, cpd_no_of_people)

In [34]:
print(cpd_location)
print(cpd_quality)
print(cpd_cost)
print(cpd_no_of_people)

╒════════════╤═════╕
│ location_0 │ 0.6 │
├────────────┼─────┤
│ location_1 │ 0.4 │
╘════════════╧═════╛
╒═══════════╤═════╕
│ quality_0 │ 0.3 │
├───────────┼─────┤
│ quality_1 │ 0.5 │
├───────────┼─────┤
│ quality_2 │ 0.2 │
╘═══════════╧═════╛
╒══════════╤════════════╤════════════╤════════════╤════════════╤════════════╤════════════╕
│ location │ location_0 │ location_0 │ location_0 │ location_1 │ location_1 │ location_1 │
├──────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ quality  │ quality_0  │ quality_1  │ quality_2  │ quality_0  │ quality_1  │ quality_2  │
├──────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ cost_0   │ 0.8        │ 0.6        │ 0.1        │ 0.6        │ 0.6        │ 0.05       │
├──────────┼────────────┼────────────┼────────────┼────────────┼────────────┼────────────┤
│ cost_1   │ 0.2        │ 0.4        │ 0.9        │ 0.4        │ 0.4        │ 0.95       │
╘══════════╧════════════╧══

#### Multiply all CPDs to obtain another CPD 
Is like a joint proba (since we have all the variables): gives the cost (first multiplied variable) **GIVEN** the other variables

In [35]:
joint_cpd = cpd_cost * cpd_location * cpd_quality * cpd_no_of_people
print(joint_cpd)
joint_cpd

╒══════════════╤═════════════════════╤════════════════╤════════════════╤════════════════╤═════════════════════╤═══════════════════════╤══════════════════════╤══════════════════════╤═════════════════════╤═════════════════════╤════════════════════════╤═══════════════════════╕
│ location     │ location_0          │ location_0     │ location_0     │ location_0     │ location_0          │ location_0            │ location_1           │ location_1           │ location_1          │ location_1          │ location_1             │ location_1            │
├──────────────┼─────────────────────┼────────────────┼────────────────┼────────────────┼─────────────────────┼───────────────────────┼──────────────────────┼──────────────────────┼─────────────────────┼─────────────────────┼────────────────────────┼───────────────────────┤
│ quality      │ quality_0           │ quality_0      │ quality_1      │ quality_1      │ quality_2           │ quality_2             │ quality_0            │ quality_0       

<TabularCPD representing P(cost:2 | location:2, quality:3, no_of_people:2) at 0x1da3c8d2a90>

#### Marginalize location and quality and observe remaining conditional probabilities
Gives cost **GIVEN** no_of_people

In [36]:
print(joint_cpd.marginalize(['location', 'quality'], inplace=False))

╒══════════════╤═════════════════════╤═════════════════════╕
│ no_of_people │ no_of_people_0      │ no_of_people_1      │
├──────────────┼─────────────────────┼─────────────────────┤
│ cost_0       │ 0.39870223503965396 │ 0.6981132075471698  │
├──────────────┼─────────────────────┼─────────────────────┤
│ cost_1       │ 0.601297764960346   │ 0.30188679245283023 │
╘══════════════╧═════════════════════╧═════════════════════╛


#### Complies with theoretical probabilities (generated with Samiam): 
![probabilities](3_c_samiam.png)

#### Obtain Joint probability by converting to a Factor

In [37]:
joint_factor = joint_cpd.to_factor()
print(joint_factor)
joint_factor

╒════════╤════════════╤═══════════╤════════════════╤═══════════════════════════════════════════╕
│ cost   │ location   │ quality   │ no_of_people   │   phi(cost,location,quality,no_of_people) │
╞════════╪════════════╪═══════════╪════════════════╪═══════════════════════════════════════════╡
│ cost_0 │ location_0 │ quality_0 │ no_of_people_0 │                                    0.0864 │
├────────┼────────────┼───────────┼────────────────┼───────────────────────────────────────────┤
│ cost_0 │ location_0 │ quality_0 │ no_of_people_1 │                                    0.0576 │
├────────┼────────────┼───────────┼────────────────┼───────────────────────────────────────────┤
│ cost_0 │ location_0 │ quality_1 │ no_of_people_0 │                                    0.1080 │
├────────┼────────────┼───────────┼────────────────┼───────────────────────────────────────────┤
│ cost_0 │ location_0 │ quality_1 │ no_of_people_1 │                                    0.0720 │
├────────┼────────────┼───────

<DiscreteFactor representing phi(cost:2, location:2, quality:3, no_of_people:2) at 0x1da3c8d23c8>

#### Marginalize location and quality and observe remaining joint probabilities
This is **joint** probabilities (contrast to above conditional probabilities). Joint probas sum to 1.

In [38]:
print(joint_factor.marginalize(['location', 'quality'], inplace=False))

╒════════╤════════════════╤══════════════════════════╕
│ cost   │ no_of_people   │   phi(cost,no_of_people) │
╞════════╪════════════════╪══════════════════════════╡
│ cost_0 │ no_of_people_0 │                   0.2212 │
├────────┼────────────────┼──────────────────────────┤
│ cost_0 │ no_of_people_1 │                   0.3108 │
├────────┼────────────────┼──────────────────────────┤
│ cost_1 │ no_of_people_0 │                   0.3336 │
├────────┼────────────────┼──────────────────────────┤
│ cost_1 │ no_of_people_1 │                   0.1344 │
╘════════╧════════════════╧══════════════════════════╛


#### Factor reduction: focus on quality=0 (good) and location=0 (good)
This is **joint** probabilities for good quality and good location

In [39]:
reduced_factor=joint_factor.reduce([('quality',0), ('location',0)], inplace=False).normalize(inplace=False)
print(reduced_factor)

╒════════╤════════════════╤══════════════════════════╕
│ cost   │ no_of_people   │   phi(cost,no_of_people) │
╞════════╪════════════════╪══════════════════════════╡
│ cost_0 │ no_of_people_0 │                   0.4800 │
├────────┼────────────────┼──────────────────────────┤
│ cost_0 │ no_of_people_1 │                   0.3200 │
├────────┼────────────────┼──────────────────────────┤
│ cost_1 │ no_of_people_0 │                   0.1600 │
├────────┼────────────────┼──────────────────────────┤
│ cost_1 │ no_of_people_1 │                   0.0400 │
╘════════╧════════════════╧══════════════════════════╛


#### Marginalize no_of_people to get the proba(cost | good quality, good location)

$$ P(cost\ |\ location=0, quality=0)$$


#### and marginalize cost to get the proba(no_of_people | good quality, good location)

$$ P(no\_of\_people\ |\ location=0, quality=0)$$

In [40]:
print(reduced_factor.marginalize(['no_of_people'], inplace=False))
print(reduced_factor.marginalize(['cost'], inplace=False))

╒════════╤═════════════╕
│ cost   │   phi(cost) │
╞════════╪═════════════╡
│ cost_0 │      0.8000 │
├────────┼─────────────┤
│ cost_1 │      0.2000 │
╘════════╧═════════════╛
╒════════════════╤═════════════════════╕
│ no_of_people   │   phi(no_of_people) │
╞════════════════╪═════════════════════╡
│ no_of_people_0 │              0.6400 │
├────────────────┼─────────────────────┤
│ no_of_people_1 │              0.3600 │
╘════════════════╧═════════════════════╛


#### Complies with theoretical probabilities  (generated with Samiam): 
![probabilities](3_d_samiam.png)

##  Inference : using Variable Elimination Algorithm
#### Use Inference to compute the above conditional probability
$$ P(cost\ |\ location=0, quality=0)$$ and $$ P(no\_of\_people\ |\ location=0, quality=0)$$

In [41]:
restaurant_inference = VariableElimination(restaurant)
inferred=restaurant_inference.query(variables=['cost', 'no_of_people'],
                           evidence={'location': 0, 'quality': 0})
print(inferred['cost'])
print(inferred['no_of_people'])

╒════════╤═════════════╕
│ cost   │   phi(cost) │
╞════════╪═════════════╡
│ cost_0 │      0.8000 │
├────────┼─────────────┤
│ cost_1 │      0.2000 │
╘════════╧═════════════╛
╒════════════════╤═════════════════════╕
│ no_of_people   │   phi(no_of_people) │
╞════════════════╪═════════════════════╡
│ no_of_people_0 │              0.6400 │
├────────────────┼─────────────────────┤
│ no_of_people_1 │              0.3600 │
╘════════════════╧═════════════════════╛


#### JointProbabilityDistribution
Like TabularCPD, JointProbabilityDistribution is a DiscreteFactor that is created from joint probabilities

In [42]:
from pgmpy.factors.discrete import JointProbabilityDistribution as JPD
joint_dist = JPD(['I','D','G'],[2,2,3], [0.126,0.168,0.126,0.009,0.045,0.126,0.252,0.0224,0.0056,0.06,0.036,0.024])
print(joint_dist)

╒═════╤═════╤═════╤════════════╕
│ I   │ D   │ G   │   P(I,D,G) │
╞═════╪═════╪═════╪════════════╡
│ I_0 │ D_0 │ G_0 │     0.1260 │
├─────┼─────┼─────┼────────────┤
│ I_0 │ D_0 │ G_1 │     0.1680 │
├─────┼─────┼─────┼────────────┤
│ I_0 │ D_0 │ G_2 │     0.1260 │
├─────┼─────┼─────┼────────────┤
│ I_0 │ D_1 │ G_0 │     0.0090 │
├─────┼─────┼─────┼────────────┤
│ I_0 │ D_1 │ G_1 │     0.0450 │
├─────┼─────┼─────┼────────────┤
│ I_0 │ D_1 │ G_2 │     0.1260 │
├─────┼─────┼─────┼────────────┤
│ I_1 │ D_0 │ G_0 │     0.2520 │
├─────┼─────┼─────┼────────────┤
│ I_1 │ D_0 │ G_1 │     0.0224 │
├─────┼─────┼─────┼────────────┤
│ I_1 │ D_0 │ G_2 │     0.0056 │
├─────┼─────┼─────┼────────────┤
│ I_1 │ D_1 │ G_0 │     0.0600 │
├─────┼─────┼─────┼────────────┤
│ I_1 │ D_1 │ G_1 │     0.0360 │
├─────┼─────┼─────┼────────────┤
│ I_1 │ D_1 │ G_2 │     0.0240 │
╘═════╧═════╧═════╧════════════╛


Reduce a JointProbabilityDistribution to focus on some observations

In [43]:
print(joint_dist.reduce([('G', 0)], inplace=False))

╒═════╤═════╤══════════╕
│ I   │ D   │   P(I,D) │
╞═════╪═════╪══════════╡
│ I_0 │ D_0 │   0.1260 │
├─────┼─────┼──────────┤
│ I_0 │ D_1 │   0.0090 │
├─────┼─────┼──────────┤
│ I_1 │ D_0 │   0.2520 │
├─────┼─────┼──────────┤
│ I_1 │ D_1 │   0.0600 │
╘═════╧═════╧══════════╛


or marginalize some variables to get probas on remaining variables

In [44]:
joint_dist.marginalize(['D'])
print(joint_dist)

╒═════╤═════╤══════════╕
│ I   │ G   │   P(I,G) │
╞═════╪═════╪══════════╡
│ I_0 │ G_0 │   0.1350 │
├─────┼─────┼──────────┤
│ I_0 │ G_1 │   0.2130 │
├─────┼─────┼──────────┤
│ I_0 │ G_2 │   0.2520 │
├─────┼─────┼──────────┤
│ I_1 │ G_0 │   0.3120 │
├─────┼─────┼──────────┤
│ I_1 │ G_1 │   0.0584 │
├─────┼─────┼──────────┤
│ I_1 │ G_2 │   0.0296 │
╘═════╧═════╧══════════╛


## Sampling
#### Sample some data from the model with some evidence
First try with rejection_sample sampler (computationally expensive)

In [45]:
from pgmpy.sampling.Sampling import BayesianModelSampling
sampler = BayesianModelSampling(restaurant)

# e.g. location=1 (bad) and cost=0 (high) should yield a P(no_people=1 (low people)) that is high
from pgmpy.factors.discrete import State
evidence = [State('location', 1), State('cost', 0)]
df=sampler.rejection_sample(evidence=evidence, size=500)
df[:5]

,location,quality,cost,no_of_people
0,1,0,0,1
1,1,1,0,1
2,1,1,0,1
3,1,1,0,1
4,1,0,0,1


In [46]:
p_people_0=len(df[df.no_of_people==0]) / float(len(df))
print('P(no_of_people=0|location=1,cost=0): %.5f' % p_people_0)

p_quality_0=len(df[df.quality==0]) / float(len(df))
print('P(quality=0|location=1,cost=0): %.5f' % p_quality_0)

P(no_of_people=0|location=1,cost=0): 0.09200
P(quality=0|location=1,cost=0): 0.35400


#### Complies with theoretical probabilities  (generated with Samiam): 
![probabilities](3_a_samiam.png)

Next try likelihood_weighted_sample (with evidence)

In [47]:
df=sampler.likelihood_weighted_sample(evidence=evidence, size=20000)
df=df.loc[:,df.columns[:-1]]
df[:5]

,location,quality,cost,no_of_people
0,1,1,0,1
1,1,1,0,0
2,1,0,0,1
3,1,0,0,1
4,1,1,0,1


In [48]:
p_people_0=len(df[df.no_of_people==0]) / float(len(df))
print('P(no_of_people=0|location=1,cost=0): %.5f' % p_people_0)

p_quality_0=len(df[df.quality==0]) / float(len(df))
print('P(quality=0|location=1,cost=0): %.5f' % p_quality_0)

P(no_of_people=0|location=1,cost=0): 0.09990
P(quality=0|location=1,cost=0): 0.29695


#### Sample some Data from the model (no evidence)
Gibbs Sampling (without evidence)

In [49]:
from pgmpy.sampling.Sampling import GibbsSampling
gibbs_sampler = GibbsSampling(restaurant)
df=gibbs_sampler.sample(size=5000)
df[:5]

,quality,no_of_people,cost,location
0,1,1,0,0
1,0,0,1,1
2,1,1,0,0
3,1,1,1,0
4,1,0,0,0


In [50]:
p_people_0=len(df[df.no_of_people==0]) / float(len(df))
print('P(no_of_people=0): %.5f' % p_people_0)

p_quality_0=len(df[df.quality==0]) / float(len(df))
print('P(quality=0): %.5f' % p_quality_0)

P(no_of_people=0): 0.55140
P(quality=0): 0.30060


Forwarder sampling (without evidence)

In [51]:
df=sampler.forward_sample(size=50000)
df[:5]

,location,quality,cost,no_of_people
0,1,1,0,1
1,1,1,1,0
2,0,0,1,0
3,0,1,0,1
4,1,1,0,1


**Verify** $$ P(no\_of\_people\ |\ location=0, quality=2)$$ 

In [52]:
p_quality_2=len(df[df.quality==2])
p_people_0_quality_2=len(df[(df.quality==2) & (df.no_of_people==0)])
p_people_0_given_quality_2 = p_people_0_quality_2 / float(p_quality_2)
print('P(people=0|quality=2): %.5f' % p_people_0_given_quality_2)

P(people=0|quality=2): 0.69679


#### Complies with theoretical probabilities (generated with Samiam): 
![probabilities](3_b_samiam.png)

## Structure Learning
### Estimate Structure with HillClimbSearch

In [53]:
from pgmpy.estimators import HillClimbSearch, BicScore,BdeuScore,K2Score

In [54]:
est = HillClimbSearch(df, scoring_method=BicScore(df))
best_model = est.estimate()
print('nodes: ' + str(best_model.nodes()))
print('edges: ' + str(best_model.edges()))

nodes: ['quality', 'no_of_people', 'cost', 'location']
edges: [('quality', 'cost'), ('cost', 'no_of_people'), ('location', 'no_of_people'), ('location', 'cost')]


In [55]:
est = HillClimbSearch(df, scoring_method=BdeuScore(df))
best_model = est.estimate()
print('nodes: ' + str(best_model.nodes()))
print('edges: ' + str(best_model.edges()))

nodes: ['quality', 'no_of_people', 'cost', 'location']
edges: [('quality', 'cost'), ('cost', 'no_of_people'), ('location', 'no_of_people'), ('location', 'cost')]


In [56]:
est = HillClimbSearch(df, scoring_method=K2Score(df))
best_model = est.estimate()
print('nodes: ' + str(best_model.nodes()))
print('edges: ' + str(best_model.edges()))

nodes: ['quality', 'no_of_people', 'cost', 'location']
edges: [('quality', 'cost'), ('cost', 'no_of_people'), ('location', 'no_of_people'), ('location', 'cost')]


### Estimate Structure with ExhaustiveSearch

In [57]:
import pandas as pd
from pgmpy.estimators import ExhaustiveSearch
df = pd.DataFrame(data=df)
s = ExhaustiveSearch(df)
df

,location,quality,cost,no_of_people
0,1,1,0,1
1,1,1,1,0
2,0,0,1,0
3,0,1,0,1
4,1,1,0,1
5,0,0,0,1
6,0,0,0,1
7,0,0,1,0
8,0,2,1,0
9,0,1,1,0


In [58]:
s = ExhaustiveSearch(df, scoring_method=BicScore(df))
for score, model in s.all_scores():
    print("{0}        {1}".format(score, model.edges()))

-154203.24119922565        [('location', 'quality')]
-154203.24119922565        [('quality', 'location')]
-154192.9573184981        []
-154099.13740838305        [('cost', 'location'), ('location', 'quality')]
-154099.13740838305        [('quality', 'location'), ('location', 'cost')]
-154099.13740838302        [('location', 'quality'), ('location', 'cost')]
-154088.8535276555        [('cost', 'location')]
-154088.85352765548        [('location', 'cost')]
-153820.72454230138        [('quality', 'location'), ('cost', 'location')]
-153677.2406927543        [('no_of_people', 'quality'), ('quality', 'location')]
-153677.2406927543        [('quality', 'no_of_people'), ('quality', 'location')]
-153677.24069275428        [('quality', 'no_of_people'), ('location', 'quality')]
-153666.95681202674        [('no_of_people', 'quality')]
-153666.95681202674        [('quality', 'no_of_people')]
-153573.1369019117        [('quality', 'no_of_people'), ('cost', 'location'), ('location', 'quality')]
-1535

## Parameter Learning
Uses above **best_model**

### MaximumLikelihoodEstimator

In [59]:
from pgmpy.estimators import MaximumLikelihoodEstimator
estimator_type = MaximumLikelihoodEstimator
best_model.fit(df, estimator=estimator_type)

In [60]:
#best_model.get_cpds()
for cpd in best_model.get_cpds():
    print('cpd for variable %s:\n%s\n' % (cpd.variable, str(cpd)))

cpd for variable cost:
╒══════════╤═════════════════════╤════════════════════╤═════════════════════╤════════════════════╤════════════════════╤═════════════════════╕
│ location │ location(0)         │ location(0)        │ location(0)         │ location(1)        │ location(1)        │ location(1)         │
├──────────┼─────────────────────┼────────────────────┼─────────────────────┼────────────────────┼────────────────────┼─────────────────────┤
│ quality  │ quality(0)          │ quality(1)         │ quality(2)          │ quality(0)         │ quality(1)         │ quality(2)          │
├──────────┼─────────────────────┼────────────────────┼─────────────────────┼────────────────────┼────────────────────┼─────────────────────┤
│ cost(0)  │ 0.7977994835522623  │ 0.6006169114195669 │ 0.10415631727107137 │ 0.5983593109105825 │ 0.6048185544336699 │ 0.05095226317091269 │
├──────────┼─────────────────────┼────────────────────┼─────────────────────┼────────────────────┼────────────────────┼──────

### BayesianEstimator

In [61]:
from pgmpy.estimators import BayesianEstimator
estimator = BayesianEstimator(best_model, df)
#for cpd in estimator.get_parameters(prior_type='dirichlet', \
#    pseudo_counts={'cost': [0.01, 0.01], 'quality': [0.01, 0.01, 0.01], \
#                   'location': [0.01, 0.01], 'no_of_people': [0.01, 0.01]}):
for cpd in estimator.get_parameters(prior_type='K2'):
    print('cpd for variable %s:\n%s\n' % (cpd.variable, str(cpd)))

cpd for variable quality:
╒════════════╤══════════╕
│ quality(0) │ 0.300042 │
├────────────┼──────────┤
│ quality(1) │ 0.49831  │
├────────────┼──────────┤
│ quality(2) │ 0.201648 │
╘════════════╧══════════╛

cpd for variable no_of_people:
╒═════════════════╤════════════════════╤════════════════════╤═════════════════════╤═════════════╕
│ cost            │ cost(0)            │ cost(0)            │ cost(1)             │ cost(1)     │
├─────────────────┼────────────────────┼────────────────────┼─────────────────────┼─────────────┤
│ location        │ location(0)        │ location(1)        │ location(0)         │ location(1) │
├─────────────────┼────────────────────┼────────────────────┼─────────────────────┼─────────────┤
│ no_of_people(0) │ 0.6056068048400624 │ 0.0978293791014639 │ 0.8012757232895437  │ 0.601953125 │
├─────────────────┼────────────────────┼────────────────────┼─────────────────────┼─────────────┤
│ no_of_people(1) │ 0.3943931951599377 │ 0.9021706208985361 │ 0.1987242767

## Scoring methods - how well data maps to a model

In [62]:
import pandas as pd

import numpy as np
from pgmpy.estimators import K2Score, BdeuScore, BicScore
# create random data sample with 3 variables, where B and C are identical:
data = pd.DataFrame(np.random.randint(0, 5, size=(5000, 2)), columns=list('AB'))
data['C'] = data['B']
model_AB_AC=BayesianModel([['A','B'], ['A','C']])
model_AB_BC=BayesianModel([['A','B'], ['B','C']])
model_BC=BayesianModel([['B','C']]) # the best model, as per data construction
model_CB=BayesianModel([['C','B']]) # the best model, as per data construction

print('K2Score AB_AC: %.5f' % K2Score(data).score(model_AB_AC))
print('K2Score AB_BC: %.5f' % K2Score(data).score(model_AB_BC))
print('K2Score BC: %.5f' % K2Score(data).score(model_BC))
print('K2Score CB: %.5f' % K2Score(data).score(model_CB))
print

print('BicScore AB_AC: %.5f' % BicScore(data).score(model_AB_AC))
print('BicScore AB_BC: %.5f' % BicScore(data).score(model_AB_BC))
print('BicScore BC: %.5f' % BicScore(data).score(model_BC))
print('BicScore CB: %.5f' % BicScore(data).score(model_CB))
print

print('BdeuScore AB_AC: %.5f' % BdeuScore(data).score(model_AB_AC))
print('BdeuScore AB_BC: %.5f' % BdeuScore(data).score(model_AB_BC))
print('BdeuScore BC: %.5f' % BdeuScore(data).score(model_BC))
print('BdeuScore CB: %.5f' % BdeuScore(data).score(model_CB))

K2Score AB_AC: -24247.54442
K2Score AB_BC: -16276.56271
K2Score BC: -8181.68648
K2Score CB: -8181.68648
BicScore AB_AC: -24310.74096
BicScore AB_BC: -16272.44038
BicScore BC: -8147.37928
BicScore CB: -8147.37928
BdeuScore AB_AC: -24269.18819
BdeuScore AB_BC: -16223.53859
BdeuScore BC: -8117.05498
BdeuScore CB: -8117.05498
